In [193]:
import json

In [194]:
vocab_raw = json.load(open('../dict/vocab_list.json', 'r'))

#clean _num_ 
num_keys = [i for i in vocab_raw.keys() if (i.find('_num_') != -1)]
for i in num_keys:
    vocab_raw.pop(i)

#new key list
vocab_keys = [u'_num_']
vocab_keys.extend(vocab_raw.keys())

#reserve index 0: padding, index 1: _num_
vocab_dict = {k:v for k, v in zip(vocab_keys, range(2, len(vocab_keys)+2))}

In [195]:
entity_raw = json.load(open('./Model/entityDict.json', 'r'))

int2entity_ = {v:k for k, v in entity_raw.items()}

int2entity = lambda x:int2entity_[x]
int2entity(1)
print(int2entity_)

{0: u'NaN', 1: u'INTERCHANGE', 2: u'SERVICE', 3: u'TITLE', 4: u'DATE', 5: u'STATION', 6: u'DURATION', 7: u'LINE', 8: u'AREA', 9: u'BUS', 10: u'LOCATION', 11: u'ROAD', 12: u'GNUM', 13: u'PERCENT', 14: u'O', 15: u'DEPOT', 16: u'PRODUCT', 17: u'NORP', 18: u'MONEY', 19: u'PERSON', 20: u'TIME', 21: u'ORG', 22: u'EVENT'}


In [196]:
pos_raw = json.load(open('./Model/posDict.json', 'r'))
print(pos_raw)
int2pos_ = {v:k for k, v in pos_raw.items()}

int2pos = lambda x:int2pos_[x]
int2pos(1)


pos2int_ = {k:v for k, v in pos_raw.items()}
print(pos2int_['ADV'])

{u'ADV': 1, u'NOUN': 2, u'NUM': 11, u'ADP': 3, u'PRON': 4, u'DET': 6, u'PROPN': 5, u'NaN': 0, u'SYM': 7, u'INTJ': 8, u'PUNCT': 10, u'PART': 9, u'X': 12, u'CONJ': 13, u'ADJ': 14, u'VERB': 15}
1


In [197]:
def word2Int(wd):
    try:
        return(vocab_dict[wd])
    except KeyError:
        return 1

print(word2Int(u'nuk'), word2Int(u'train'))

(1, 2063)


In [198]:
def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence

    l :: array containing the word indexes

    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >= 1
    l = list(l)

    lpadded = win // 2 * [-1] + l + win // 2 * [-1]
    out = [lpadded[i:(i + win)] for i in range(len(l))]

    assert len(out) == len(l)
    return out

In [222]:
from textacy import Doc, preprocess

input_text = u'In 2016 trains travelled an average 174000 trainkilometres between delays of more than five minutes up'

test_text = Doc(preprocess.replace_numbers(input_text, replace_with='_num_'))
tag_list = [pos2int_[tag[1]] for tag in  test_text.pos_tagged_text[0]]
array_text = [word2Int(i.lemma_) for i in test_text]

In [223]:
maxlen = 3
TEST_X = contextwin(array_text, maxlen)
TEST_P = contextwin(tag_list, maxlen)

In [224]:
TEST_P

[[-1, 3, 14],
 [3, 14, 2],
 [14, 2, 2],
 [2, 2, 2],
 [2, 2, 15],
 [2, 15, 6],
 [15, 6, 14],
 [6, 14, 14],
 [14, 14, 2],
 [14, 2, 2],
 [2, 2, 2],
 [2, 2, 3],
 [2, 3, 2],
 [3, 2, 3],
 [2, 3, 14],
 [3, 14, 3],
 [14, 3, 11],
 [3, 11, 2],
 [11, 2, 9],
 [2, 9, -1]]

In [225]:
TEST_X

[[-1, 457, 1],
 [457, 1, 1],
 [1, 1, 1],
 [1, 1, 2063],
 [1, 2063, 135],
 [2063, 135, 2382],
 [135, 2382, 992],
 [2382, 992, 1],
 [992, 1, 1],
 [1, 1, 1],
 [1, 1, 1],
 [1, 1, 1050],
 [1, 1050, 2101],
 [1050, 2101, 2073],
 [2101, 2073, 1420],
 [2073, 1420, 943],
 [1420, 943, 2464],
 [943, 2464, 683],
 [2464, 683, 1917],
 [683, 1917, -1]]

In [226]:
import keras
from keras.models import model_from_json

def loadModel():
    loadfile = open("./Model/model-keras1.json", 'r')
    model = model_from_json(loadfile.read())
    model.load_weights("./Model/model-keras1.h5")
    return model

ner_model = loadModel()

In [227]:
import numpy as np

#print(map(lambda x:x.append('haha'),TEST_X))
#print(TEST_X)
TEST_X_NP = np.array(TEST_X)

TEST_X_NP[TEST_X_NP == -1] = 0
##print(TEST_X_NP.shape)
#print(dir(ner_model))

POS = np.array(TEST_P)
POS[POS == -1] = 0
print(POS)
v = ner_model.predict({'word_input':TEST_X_NP,'pos_input':POS})

##print(int2entity_[0])
predicted_entity_id = v.argmax(axis=1)
predicted_entity = [int2entity_[x] for x in predicted_entity_id]
for i in zip(Doc(input_text), predicted_entity):
    print i,

[[ 0  3 14]
 [ 3 14  2]
 [14  2  2]
 [ 2  2  2]
 [ 2  2 15]
 [ 2 15  6]
 [15  6 14]
 [ 6 14 14]
 [14 14  2]
 [14  2  2]
 [ 2  2  2]
 [ 2  2  3]
 [ 2  3  2]
 [ 3  2  3]
 [ 2  3 14]
 [ 3 14  3]
 [14  3 11]
 [ 3 11  2]
 [11  2  9]
 [ 2  9  0]]
(In, u'O') (2016, u'O') (trains, u'O') (travelled, u'O') (an, u'O') (average, u'O') (174000, u'O') (trainkilometres, u'O') (between, u'O') (delays, u'O') (of, u'O') (more, u'O') (than, u'O') (five, u'O') (minutes, u'O') (up, u'O')
